In [16]:
import os
import sys
import numpy as np
from skimage import io
import SimpleITK as sitk
from skimage.filters import gaussian

In [2]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data'
moving_path = os.path.join(DATA, 'MD585/preps/CH1/moving_volume.tif')
fixed_path = os.path.join(DATA, 'MD589/preps/CH1/moving_volume.tif')
print(fixed_path)

/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/MD589/preps/CH1/moving_volume.tif


In [45]:
def normalize8(img):
    mn = img.min()
    mx = img.max()
    mx -= mn
    img = ((img - mn)/mx) * 2**8 - 1
    return np.round(img).astype(np.uint8) 

def pad_volume(size, volume):
    size_difference = size - volume.shape
    xr, yr, zr = ((size_difference)/2).astype(int)
    xl, yl, zl = size_difference - np.array([xr, yr, zr])
    return np.pad(volume, [[xl, xr], [yl, yr], [zl, zr]])


In [3]:
fixed_image = sitk.ReadImage(fixed_path, sitk.sitkFloat32)
moving_image = sitk.ReadImage(moving_path, sitk.sitkFloat32)

In [4]:
iterationNumbers = 600
spatialSamples = 6000
simpleElastix = sitk.ElastixImageFilter()
simpleElastix.SetParameterMap(sitk.GetDefaultParameterMap('translation'))
simpleElastix.AddParameterMap(sitk.GetDefaultParameterMap('affine'))
simpleElastix.AddParameterMap(sitk.GetDefaultParameterMap('bspline'))
simpleElastix.SetParameter("Registration","MultiMetricMultiResolutionRegistration")
simpleElastix.SetParameter( "Metric", ("NormalizedMutualInformation", "CorrespondingPointsEuclideanDistanceMetric",))
simpleElastix.SetParameter("Metric0Weight", "0.0")
simpleElastix.SetParameter("MaximumNumberOfIterations" , str(iterationNumbers))
simpleElastix.SetParameter("NumberOfSpatialSamples" , str(spatialSamples))
#simpleElastix.PrintParameterMap()

<SimpleITK.SimpleITK.ElastixImageFilter; proxy of <Swig Object of type 'itk::simple::ElastixImageFilter::Self *' at 0x7fa36612cc00> >

In [5]:
%%time
simpleElastix.SetFixedImage(fixed_image)
simpleElastix.SetMovingImage(moving_image)
simpleElastix.SetOutputDirectory('/tmp')
simpleElastix.SetFixedPointSetFileName("/tmp/fixed_points.pts")
simpleElastix.SetMovingPointSetFileName("/tmp/moving_points.pts")
simpleElastix.LogToConsoleOff()
resultImage = simpleElastix.Execute()

CPU times: user 13min 3s, sys: 7.93 s, total: 13min 11s
Wall time: 9min 53s


In [6]:
img = sitk.GetArrayFromImage(resultImage)
type(img), img.dtype, img.shape

(numpy.ndarray, dtype('float32'), (447, 253, 342))

In [7]:
savepath = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/MD585/preps/CH1/registered_585.tif'
io.imsave(savepath, img)

In [161]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
MD585_volumepath = os.path.join(DATA, 'MD585_MD589.tif')
MD589_volumepath = os.path.join(DATA, 'MD589_25um_sagittal.tif')
MD594_volumepath = os.path.join(DATA, 'MD594_MD589.tif')
population = [MD585_volumepath, MD589_volumepath, MD594_volumepath]

In [156]:
MD585_volume = io.imread(MD585_volumepath)
MD589_volume = io.imread(MD589_volumepath)
MD594_volume = io.imread(MD594_volumepath)
volumes = [MD585_volume, MD589_volume.astype(np.float32), MD594_volume]
volumes2 = [MD585_volume, MD594_volume]

In [148]:
MD585_volume.dtype, MD589_volume.dtype, MD594_volume.dtype

(dtype('float32'), dtype('uint8'), dtype('float32'))

In [153]:
merged_volume = np.sum(volumes, axis=0)
print(f'post max={np.max(merged_volume)} mean={np.mean(merged_volume)}')
arr = normalize8(merged_volume)

post max=803.9765625 mean=79.67485046386719


In [154]:
arr.dtype, arr.shape

(dtype('uint8'), (447, 253, 341))

In [155]:
savepath = os.path.join(DATA, 'gauss.tif')
io.imsave(savepath, arr)

In [163]:
# Concatenate the ND images into one (N+1)D image
vectorOfImages = sitk.VectorOfImage()

for filename in population:
    print(filename)
    vectorOfImages.push_back(sitk.ReadImage(filename, sitk.sitkFloat32))

image = sitk.JoinSeries(vectorOfImages)

# Register
elastixImageFilter = sitk.ElastixImageFilter()
elastixImageFilter.SetFixedImage(image)
elastixImageFilter.SetMovingImage(image)
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap('groupwise'))
elastixImageFilter.Execute()

/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/MD585_MD589.tif
/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/MD589_25um_sagittal.tif
/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/MD594_MD589.tif
ELASTIX version: 5.000
Command line options from ElastixBase:
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ./
-threads  unspecified, so all available threads are used
  The default value "true" is used instead.

  From elastix 4.8 it defaults to true!
This may change the behavior of your registrations considerably.

Command line options from TransformBase:
-t0       unspecified, so no initial transform used
  The default value "3" is used instead.

Reading images...
Reading images took 0 ms.

  A default pyramid schedule is used.
  A default pyramid schedule is used.
Initialization of all components (before registration) took: 2 ms.

itk::ExceptionObject (0x5580f97e75e0)
Loca

RuntimeError: Exception thrown in SimpleITK ElastixImageFilter_Execute: /src/SimpleElastix/Code/Elastix/src/sitkElastixImageFilterImpl.cxx:259:
sitk::ERROR: 
itk::ExceptionObject (0x5580f97e75e0)
Location: "unknown" 
File: /src/SimpleElastix/build/Elastix/Core/Main/elxElastixFilter.hxx
Line: 253
Description: itk::ERROR: ElastixFilter(0x5580ea6eaff0): Internal elastix error: See elastix log (use LogToConsoleOn() or LogToFileOn()).

